In [42]:
import random # for choosing random word from lexicon
from collections import defaultdict # for creating letter frequency maps
import re # for regex

In [33]:
# Simulate hangman 

def generate_string(correct,guessed_letter,res): 
    guessed_correct = False
    for i in range(len(correct)): 
        if correct[i]==guessed_letter: 
            res[i]=guessed_letter
            guessed_correct = True
    return guessed_correct

def simulate_hangman(correct,guesses):
    res = ['_']*len(correct)
    guessed_correct = False
    while guesses: 
        print('Word to guess: ',res) 
        guess = input('Enter guess: ') 
        if guess in res: 
            print('Letter already revealed. Enter a different one.')
        guessed_correct_letter = generate_string(correct,guess,res)
        if ''.join(res) == correct:
            guessed_correct = True 
            break
        if not guessed_correct_letter:
            guesses -= 1
        print('Guesses remaining: ',guesses)
    print('Correct word: ',correct)
    if guessed_correct: 
        print('Nice! You guessed correctly!')
    else:
        print('Oops! Better luck next time.') 

In [9]:
lexicon = ["apple", "banana", "orange", "grape", "strawberry", "pineapple", "mango", "kiwi"]
correct = random.choice(lexicon)
guesses = 5 

simulate_hangman(correct,guesses)

Word to guess:  ['_', '_', '_', '_', '_']


Enter guess:  a


Word to guess:  ['_', '_', 'a', '_', '_']


Enter guess:  g


Word to guess:  ['g', '_', 'a', '_', '_']


...

Word to guess:  ['g', '_', 'a', 'p', '_']


Enter guess:  r


Word to guess:  ['g', 'r', 'a', 'p', '_']


Enter guess:  e


Correct word:  grape
Nice! You guessed correctly!


In [11]:
# lexicon of five letter English words
lexicon = []

with open('./words.txt') as f: 
    try:
        for line in f:
            word = line.strip().lower() 
            lexicon.append(word)
    except FileNotFoundError:
        print('ERROR: File not found.')

print('Lexicon created successfully. Number of words in lexicon: ',len(lexicon))

Lexicon created successfully. Number of words in lexicon:  5757


In [26]:
# create letter frequency map 
letter_frequency = defaultdict(int)
for word in lexicon: 
    for letter in word:
        if('a' <= letter <= 'z'): 
            letter_frequency[letter] += 1

In [29]:
print('Letter frequencies:')
for letter,count in sorted(letter_frequency.items(), key=lambda x:x[1],reverse=True): 
    print(f'{letter}: {count}')

Letter frequencies:
s: 3033
e: 3009
a: 2348
o: 1915
r: 1910
i: 1592
l: 1586
t: 1585
n: 1285
d: 1181
u: 1089
c: 964
p: 955
y: 886
m: 843
h: 814
b: 715
g: 679
k: 596
f: 561
w: 505
v: 318
x: 139
z: 135
j: 89
q: 53


In [34]:
# Lets test hangman game on this newly created lexicon 
correct = random.choice(lexicon) 
guesses = 5 

simulate_hangman(correct,guesses) 

Word to guess:  ['_', '_', '_', '_', '_']


Enter guess:  s


Guesses remaining:  5
Word to guess:  ['_', '_', '_', '_', 's']


Enter guess:  r


Guesses remaining:  5
Word to guess:  ['_', '_', 'r', '_', 's']


...

Guesses remaining:  2
Word to guess:  ['_', '_', 'r', '_', 's']


Enter guess:  m


Guesses remaining:  1
Word to guess:  ['_', '_', 'r', '_', 's']


Enter guess:  k


Guesses remaining:  0
Correct word:  dirts
Oops! Better luck next time.


Okay, now on to the real stuff. 
First, we'll try the binary search approach. Each turn, our guess will be the letter which occurs in close to half of the possible words. 
Based on the outcome, we eliminate the other half, and consider all words which fit the outcome (like regex). 

In [40]:
lexicon_size = len(lexicon)
letter_frequency_fractions = {letter: count/lexicon_size for letter, count in letter_frequency.items()}

print('Fraction of words containing a letter: ') 
for k,v in sorted(letter_frequency_fractions.items(),key=lambda x:x[1],reverse=True): 
    print(f'{k}: {v:.4f}')

Fraction of words containing a letter: 
s: 0.5268
e: 0.5227
a: 0.4079
o: 0.3326
r: 0.3318
i: 0.2765
l: 0.2755
t: 0.2753
n: 0.2232
d: 0.2051
u: 0.1892
c: 0.1674
p: 0.1659
y: 0.1539
m: 0.1464
h: 0.1414
b: 0.1242
g: 0.1179
k: 0.1035
f: 0.0974
w: 0.0877
v: 0.0552
x: 0.0241
z: 0.0234
j: 0.0155
q: 0.0092


In [50]:
# find letter frequencies after each guess 
def calculate_letter_frequencies(res,incorrect_guesses,lexicon): 
    letter_frequency = defaultdict(int)
    
    # create regex from res 
    regex = []
    for char in res:
        if char == '_':
            if incorrect_guesses:
                escaped_incorrect = ''.join(re.escape(g) for g in incorrect_guesses)
                regex.append(f"[^{escaped_incorrect}]")
            else:
                regex.append('.')
        else:
            regex.append(re.escape(char))
    regex = '^' + ''.join(regex) + '$'

    # create letter frequency map out of possible words
    lexicon_size = 0
    for word in lexicon:
        if re.fullmatch(regex,word): 
            for letter in word:
                if('a' <= letter <= 'z'): 
                    letter_frequency[letter] += 1
            lexicon_size += 1
       
    
    # convert frequency to fraction
    if lexicon_size > 0:
        letter_frequency_fractions = {letter: count/lexicon_size for letter, count in letter_frequency.items()}
    
    return letter_frequency

In [51]:
def guess(res,guessed_letters,incorrect_guesses,lexicon): 
    # based on res, update letter frequencies 
    letter_frequencies = calculate_letter_frequencies(res,incorrect_guesses,lexicon)
    
    # find the letter which has frequency closest to 0.5
    max_diff = 0.5 
    guessed_letter = 'a' # random guess
    for letter in letter_frequencies: 
        # skip already guessed letters
        if letter in guessed_letters: 
            continue
        diff = abs(letter_frequencies[letter] - 0.5)
        if diff < max_diff: 
            guessed_letter = letter 
            max_diff = diff

    guessed_letters.append(guessed_letter)
            
    return guessed_letter

In [54]:
def simulate_hangman_v2(correct,guesses): 
    res = ['_'] * len(correct) 
    guessed_letters = []
    incorrect_guesses = []
    guessed_correct = False
    
    print(f'You have {guesses} guesses. You will lose a guess each time you guess incorrectly.')
    
    while guesses: 
        print('Word to guess: ',res) 
        guessed_letter = guess(res,guessed_letters,incorrect_guesses,lexicon) 
        guessed_correct_letter = generate_string(correct,guess,res)
        if ''.join(res) == correct:
            guessed_correct = True 
            break
        if not guessed_correct_letter:
            incorrect_guesses.append(guessed_letter)
            guesses -= 1
        print('Guesses remaining: ',guesses)
        
    print('Correct word: ',correct)
    
    if guessed_correct: 
        print('Nice! You guessed correctly!')
    else:
        print('Oops! Better luck next time.') 

In [55]:
correct = random.choice(lexicon)
guesses = 5

simulate_hangman_v2(correct,guesses)

You have 5 guesses. You will lose a guess each time you guess incorrectly.
Word to guess:  ['_', '_', '_', '_', '_']
Guesses remaining:  4
Word to guess:  ['_', '_', '_', '_', '_']
Guesses remaining:  3
Word to guess:  ['_', '_', '_', '_', '_']
Guesses remaining:  2
Word to guess:  ['_', '_', '_', '_', '_']
Guesses remaining:  1
Word to guess:  ['_', '_', '_', '_', '_']
Guesses remaining:  0
Correct word:  selah
Oops! Better luck next time.
